In [45]:
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

In [46]:
data = np.loadtxt('../data/MacroSeries.txt', delimiter=',')

In [47]:
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]

In [48]:
def get_z_w(alpha, k, w):
    n = k.shape[0]
    z = np.empty(n)
    for i in range(n):
        z[i] = np.log(w[i] / ((1 - alpha) * (k[i] ** alpha)))
    return z

In [49]:
def get_p(rho, mu, sigma, z):
    n = z.shape[0]
    p = np.empty(n)
    for i in range(n):
        if i == 0:
            p[i] = sts.norm.pdf(z[i] - mu, 0, sigma)
        else:
            p[i] = sts.norm.pdf(z[i] - (rho * z[i - 1]) - ((1 - rho) * mu), 0, sigma)
    p[p == 0] = 1e-10
    return p

In [50]:
def log_lik_w(alpha, rho, mu, sigma, k, w):
    z = get_z_w(alpha, k, w)
    p = get_p(rho, mu, sigma, z)
    ln_pdf_vals = np.log(p)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

In [51]:
def crit_w(params, k, w):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_w(alpha, rho, mu, sigma, k, w)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

In [52]:
alpha_init = 0.5
rho_init = 0.7
mu_init = 9.5
sigma_init = 0.1
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
results_w = opt.minimize(crit_w, params_init, args=(k, w), bounds=((1e-10, 1), (-1, 1), (1e-10, None), (1e-10, None)))
alpha_MLE_w, rho_MLE_w, mu_MLE_w, sigma_MLE_w = results_w.x
print('alpha_MLE_w: ', alpha_MLE_w, 'rho_MLE_w: ', rho_MLE_w, 'mu_MLE_w: ', mu_MLE_w, 'sigma_MLE: ', sigma_MLE_w)
print('Maximum log likelihood: {}'.format(log_lik_w(alpha_MLE_w, rho_MLE_w, mu_MLE_w, sigma_MLE_w, k, w)))
print('Inverse Hessian variance-covariance matrix: \n{}'.format(results_w.hess_inv.todense()))

alpha_MLE_w:  0.459335309273 rho_MLE_w:  0.719165661591 mu_MLE_w:  9.4972894385 sigma_MLE:  0.0919962260786
Maximum log likelihood: 96.70687671414075
Inverse Hessian variance-covariance matrix: 
[[ 0.00632257  0.02115401 -0.07350399  0.00124115]
 [ 0.02115401  1.14401947 -0.02324772  0.04824339]
 [-0.07350399 -0.02324772  0.99847788 -0.00337276]
 [ 0.00124115  0.04824339 -0.00337276  0.00214132]]


In [53]:
def get_z_r(alpha, k, r):
    n = k.shape[0]
    z = np.empty(n)
    for i in range(n):
        z[i] = np.log(r[i] / (alpha * (k[i] ** (alpha - 1))))
    return z

In [54]:
def log_lik_r(alpha, rho, mu, sigma, k, r):
    z = get_z_r(alpha, k, r)
    p = get_p(rho, mu, sigma, z)
    ln_pdf_vals = np.log(p)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

In [55]:
def crit_r(params, k, r):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_r(alpha, rho, mu, sigma, k, r)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

In [56]:
results_r = opt.minimize(crit_r, params_init, args=(k, r), bounds=((1e-10, 1), (-1, 1), (1e-10, None), (1e-10, None)))
alpha_MLE_r, rho_MLE_r, mu_MLE_r, sigma_MLE_r = results_r.x
print('alpha_MLE_r: ', alpha_MLE_r, 'rho_MLE_r: ', rho_MLE_r, 'mu_MLE_r: ', mu_MLE_r, 'sigma_MLE_r: ', sigma_MLE_r)
print('Maximum log likelihood: {}'.format(log_lik_r(alpha_MLE_r, rho_MLE_r, mu_MLE_r, sigma_MLE_r, k, w)))
print('Inverse Hessian variance-covariance matrix: \n{}'.format(results_r.hess_inv.todense()))

alpha_MLE_r:  0.450217961795 rho_MLE_r:  0.725725702046 mu_MLE_r:  9.50188499049 sigma_MLE_r:  0.0919967237379
Maximum log likelihood: -2302.5850929940457
Inverse Hessian variance-covariance matrix: 
[[  4.18268693e-03   4.33577277e-02  -5.42768067e-02   6.32800281e-04]
 [  4.33577277e-02   1.69983962e+00   3.64157376e-02   2.15538442e-02]
 [ -5.42768067e-02   3.64157376e-02   9.97049341e-01  -6.98595327e-04]
 [  6.32800281e-04   2.15538442e-02  -6.98595327e-04   5.20886403e-04]]


In [61]:
z_star = np.log(1 / (alpha_MLE_w * (7500000 ** (alpha_MLE_w - 1))))
p_star = 1 - sts.norm.cdf(z_star - (rho_MLE_w * 10) - ((1 - rho_MLE_w) * mu_MLE_w), 0, sigma_MLE_w)
p_star

0.99999999298700304